In [ ]:
import json
import os
import math

import utils.instrument as instru

with open('./config/scpi_commands.json') as f:
    scpi_cmds = json.load(f)['scpi_cmds']

ip_addr = r'192.168.0.1'

sa = instru.SpectrumAnalyzer(scpi_cmds, ip_addr)

Connected to: Rohde&Schwarz,FSQ-26,200750/026,4.75 successfully.


In [6]:
def unit_measurement(sa: instru.SpectrumAnalyzer, sa_config: dict):
    sa.config(param='continues_sweep', value='OFF')  # stop sweep
    input('Waiting for board launch up. Press enter to start measurement.')

    sa.config(param=sa_config)  # start continues sweep

    input(f"Waiting trace to stabilize. Press Enter to stop sweep.")
    sa.config(param='continues_sweep', value='OFF')  # stop sweep

    peak_freq, peak_power = sa.aquire_peak_point()

    return peak_freq, peak_power

def search(sa: instru.SpectrumAnalyzer, giteki_dict: dict, measure_item: str, freq_band: str,
           working_dir: str, rule=r'49_27_3', method=r'general', spurious=True):
    
    """
    A function to perform 'search' and 'zoom in' step
    """
    screenshots_save_dir = os.path.join(working_dir, 'screenshots')
    trace_data_save_dir = os.path.join(working_dir, 'trace_data')

    # search step
    sa_config = giteki_dict['search']['common']
    sa_config.update(giteki_dict['masks'][rule][freq_band]['freq'])  # add freq info
    if measure_item == 'peak':
        sa_config.update(giteki_dict['search']['diff']['peak']['general'])

    search_freq, search_peak = unit_measurement(sa, sa_config)  # perform a measurement
    screenshot_name = f"{measure_item}_search.png"
    sa.save_screenshot(os.path.join(screenshots_save_dir, screenshot_name))

    # judge if is spurious measurement according to freq_band

    if measure_item == 'average':
        trace_data_name = f'trace_data_interval({freq_band}).csv'
        sa.save_trace_to_csv(os.path.join(trace_data_save_dir, trace_data_name))

    del sa_config['start_freq']
    del sa_config['stop_freq']
    if measure_item == 'peak':
        spans = ['100MHz']
        sa_config.update(giteki_dict['search']['diff']['peak'][method])
    else:
        spans = ['100MHz', '10MHz']

    for span in spans:
        sa_config.update({'span': span, 'center_freq': search_freq})
        search_freq, search_peak = unit_measurement(sa, sa_config)  # perform a measurement
        screenshot_name = f"{measure_item}_search_{span}.png"
        sa.save_screenshot(os.path.join(screenshots_save_dir, screenshot_name))

    if not spurious and measure_item == 'peak':  # add correction factor
        rbw = float(giteki_dict['search']['diff']['peak'][method]['RBW'].split('MHz')[0])
        search_peak += 20 * math.log10(float(50.0) / rbw)

    return (search_freq, search_peak)

def determination(result: tuple, giteki_dict: dict, measure_item: str, freq_band: str, rule):
    freq, peak = result
    limit_dict = giteki_dict['masks'][rule][freq_band]['limit']

    if measure_item not in limit_dict:
        dividing_freq = float(limit_dict['dividing_freq'].split('GHz')[0]) * 1e9
        measure_item = 'average1' if freq < dividing_freq else 'average2'
    limit = float(limit_dict[measure_item].split('dBm')[0])

    if 'allowable_dev' in limit_dict:
        allowable_dev = float(limit_dict['allowable_dev'].split('%')[0]) / 100
        limit = 10 * math.log10((10 ** (limit / 10)) * (1 + allowable_dev))

    status = 'Passed' if peak <= limit else "Failed"

    return {
        f'freq': f"{round(freq / 1e9, 5)}GHz",
        f'{measure_item}': [f"{round(peak, 2)}dBm", status],
    }


In [17]:
freq_band = '7.587GHz~8.4GHz'

with open('./config/giteki_tmp.json') as f:
    giteki_dict = json.load(f)


search(sa, giteki_dict, measure_item='peak', freq_band=freq_band, working_dir='', spurious=False)


Screenshot saved.
Screenshot saved.


(8058402000.0, -25.953638563581077)

In [22]:
res = determination((8058402000.0, -25.953638563581077), giteki_dict, 'peak', freq_band, rule='49_27_3')